In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Querying the bronze table

In [0]:
df = spark.read.table('sentimental_analysis.bronze.social_media_bronze')
display(df.limit(5))

id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time
6756104069413467992,Phishing scam stole customer data. Else dark require let own father itself. #Ransomware @taylortimothy,2025-01-25T07:22:25.000Z,@barajaslucas,26048909,fr,2,7,0,2,47,null,null,Android,false,false,null,7969315299031362774,17585,1377,false,New Kevinport,Phishing,social_engineering,database,2025-12-26T18:16:01.467Z
4698209966954012889,Administration company require beautiful happen authority whom. #Phishing @christopherhudson @lisabrooks,2024-12-31T06:39:35.000Z,@obarnes,249928258,en,5,4,1,0,45,https://example.com/news/62,https://img.example.com/5.jpg,TweetDeck,false,true,null,9211089253166534054,9534,920,false,South Garyville,null,unknown,null,2025-12-26T18:16:01.467Z
3409203360084917437,Indeed bank budget find parent listen head door operation deal professor. #DataBreach @robertbruce @erica16,2025-02-25T22:00:54.000Z,@courtneyfox,832428340,es,3,10,1,2,45,null,https://img.example.com/4.jpg,Android,false,false,null,8938789069609500589,11194,100,false,null,null,unknown,null,2025-12-26T18:16:01.467Z
5509752392425025109,Rate lead on form while national arrive best travel. #ThreatIntel #MFA #DataBreach @knightanne @karathompson,2024-08-17T03:08:47.000Z,@elizabeth72,45483312,fr,1,5,1,1,44,null,null,TweetDeck,false,false,null,2309839474879821148,27874,1024,false,null,null,unknown,null,2025-12-26T18:16:01.467Z
1038570393285133048,Great server update! Specific have maintain yard sister ok guy analysis. #Ransomware @owilliams,2024-12-28T22:15:04.000Z,@wevans,305345870,en,4,10,2,1,49,null,null,Twitter for iPhone,false,false,null,2423854976071670313,2691,671,false,Adamburgh,Brute Force,credential,server,2025-12-26T18:16:01.467Z


### Removing Duplicates Row based on all columns

In [0]:
print(f"Before deduplicating {df.count()}")
df = df.dropDuplicates(['id', "text", 'created_at'])
print(f"After deduplicating {df.count()}")

Before deduplicating 503456
After deduplicating 503456


### Dropping rows if id, text or created_at is null

In [0]:
df = df.filter('(id is not null) and (text is not null) and (created_at is not null)')

### Checking for duplicate id's

In [0]:
df.groupby('id').agg(count('*').alias('count')).filter('count > 1').display()

id,count


### Handling Null Values

In [0]:
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])

display(null_counts)


id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time
0,0,0,0,0,0,0,0,0,0,0,352548,402860,0,0,0,402642,0,0,0,0,150971,301716,0,301716,0


In [0]:
df = df.fillna("unknown", ["username", "user_id", "language", "urls", "media_urls", "source", "user_location", "attack_type", "delivery_method", "context_target"])
df = df.fillna(0, ["retweet_count", "like_count", "reply_count", "quote_count", "impression_count", "user_followers_count", "user_following_count"])
df.limit(5).display()

id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time
2501625083271970972,Reviewing database logs. #Phishing #CyberSecurity #DDoS @mccannkimberly,2025-01-16T04:01:48.000Z,@laurenmathis,870121632,en,2,5,0,0,48,unknown,unknown,Android,false,false,null,3853029650318035750,3451,219,false,New Sandraborough,Credential Stuffing,credential,database,2025-12-26T18:16:01.467Z
5051073293040626730,New phishing campaign detected. Doctor sister owner structure allow. #CyberSecurity #CVE @thomasjones @fwilliams,2024-11-08T13:08:12.000Z,@smclaughlin,753385439,es,4,5,1,0,51,https://example.com/news/86,unknown,Android,false,false,null,1074380751614937220,180504,752,false,South William,Phishing,social_engineering,web_portal,2025-12-26T18:16:01.467Z
3820283122683259858,Account concern present expect experience stay throughout large model move morning commercial. #DDoS,2024-12-15T08:00:50.000Z,@garyperez,429061271,en,4,7,2,0,59,unknown,unknown,Android,false,false,null,6976430402798092748,1871,1481,false,West Sheilaberg,unknown,unknown,unknown,2025-12-26T18:16:01.467Z
8373494114952244489,Lawyer area change data hair tonight theory two per. You contain end. #Ransomware #DDoS #DataBreach @wmartinez @julie16,2025-02-17T21:20:29.000Z,@dnelson,872172330,en,3,8,0,2,44,unknown,unknown,Twitter Web App,false,false,null,4724040809333899990,1819,1176,false,Alyssastad,unknown,unknown,unknown,2025-12-26T18:16:01.467Z
1967683443240152632,Place those peace interesting house course Republican mouth evening part person practice. #CVE #DataBreach @anthonycastaneda @woodkyle,2024-12-22T12:45:37.000Z,@matthewjones,69631146,en,0,10,1,1,58,unknown,https://img.example.com/3.jpg,Android,false,false,null,9618597534815003393,4300,1994,false,unknown,unknown,unknown,unknown,2025-12-26T18:16:01.467Z


In [0]:
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])

display(null_counts)


id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,402642,0,0,0,0,0,0,0,0,0


### creating new column for the extraction

In [0]:
df = df.withColumn(
    "tokens",
    split(col("text"), " ")
)

### Filtering the hashtags and mentions in the tokens column

In [0]:
df = df.withColumn("hashtags_array",expr("filter(tokens, x -> x LIKE '#%')"))\
    .withColumn("mentions_array", expr("filter(tokens, x -> x LIKE '@%')"))

df.limit(5).display()

id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time,tokens,hashtags_array,mentions_array
2501625083271970972,Reviewing database logs. #Phishing #CyberSecurity #DDoS @mccannkimberly,2025-01-16T04:01:48.000Z,@laurenmathis,870121632,en,2,5,0,0,48,unknown,unknown,Android,false,false,null,3853029650318035750,3451,219,false,New Sandraborough,Credential Stuffing,credential,database,2025-12-26T18:16:01.467Z,"List(Reviewing, database, logs., #Phishing, #CyberSecurity, #DDoS, @mccannkimberly)","List(#Phishing, #CyberSecurity, #DDoS)",List(@mccannkimberly)
5051073293040626730,New phishing campaign detected. Doctor sister owner structure allow. #CyberSecurity #CVE @thomasjones @fwilliams,2024-11-08T13:08:12.000Z,@smclaughlin,753385439,es,4,5,1,0,51,https://example.com/news/86,unknown,Android,false,false,null,1074380751614937220,180504,752,false,South William,Phishing,social_engineering,web_portal,2025-12-26T18:16:01.467Z,"List(New, phishing, campaign, detected., Doctor, sister, owner, structure, allow., #CyberSecurity, #CVE, @thomasjones, @fwilliams)","List(#CyberSecurity, #CVE)","List(@thomasjones, @fwilliams)"
3820283122683259858,Account concern present expect experience stay throughout large model move morning commercial. #DDoS,2024-12-15T08:00:50.000Z,@garyperez,429061271,en,4,7,2,0,59,unknown,unknown,Android,false,false,null,6976430402798092748,1871,1481,false,West Sheilaberg,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Account, concern, present, expect, experience, stay, throughout, large, model, move, morning, commercial., #DDoS, )",List(#DDoS),List()
8373494114952244489,Lawyer area change data hair tonight theory two per. You contain end. #Ransomware #DDoS #DataBreach @wmartinez @julie16,2025-02-17T21:20:29.000Z,@dnelson,872172330,en,3,8,0,2,44,unknown,unknown,Twitter Web App,false,false,null,4724040809333899990,1819,1176,false,Alyssastad,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Lawyer, area, change, data, hair, tonight, theory, two, per., You, contain, end., #Ransomware, #DDoS, #DataBreach, @wmartinez, @julie16)","List(#Ransomware, #DDoS, #DataBreach)","List(@wmartinez, @julie16)"
1967683443240152632,Place those peace interesting house course Republican mouth evening part person practice. #CVE #DataBreach @anthonycastaneda @woodkyle,2024-12-22T12:45:37.000Z,@matthewjones,69631146,en,0,10,1,1,58,unknown,https://img.example.com/3.jpg,Android,false,false,null,9618597534815003393,4300,1994,false,unknown,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Place, those, peace, interesting, house, course, Republican, mouth, evening, part, person, practice., #CVE, #DataBreach, @anthonycastaneda, @woodkyle)","List(#CVE, #DataBreach)","List(@anthonycastaneda, @woodkyle)"


### Cleaning the hashtags_array and mentions_array column

In [0]:
# Remove '#' from hashtags and '@' from mentions by taking substring from index 1
df = df.withColumn(
    "hashtags_array",
    expr("transform(hashtags_array, x -> substring(x, 2))")
).withColumn(
    "mentions_array",
    expr("transform(mentions_array, x -> substring(x, 2))")
).withColumn(
    # Remove hashtags and mentions from text to create cleaned_text
    "cleaned_text",
    expr("""
        concat_ws(
            ' ',
            filter(tokens, x -> NOT (x LIKE '#%' OR x LIKE '@%'))
        )
    """)
)

df.limit(5).display()

id,text,created_at,username,user_id,language,retweet_count,like_count,reply_count,quote_count,impression_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,user_followers_count,user_following_count,user_verified,user_location,attack_type,delivery_method,context_target,ingest_time,tokens,hashtags_array,mentions_array,cleaned_text
2501625083271970972,Reviewing database logs. #Phishing #CyberSecurity #DDoS @mccannkimberly,2025-01-16T04:01:48.000Z,@laurenmathis,870121632,en,2,5,0,0,48,unknown,unknown,Android,false,false,null,3853029650318035750,3451,219,false,New Sandraborough,Credential Stuffing,credential,database,2025-12-26T18:16:01.467Z,"List(Reviewing, database, logs., #Phishing, #CyberSecurity, #DDoS, @mccannkimberly)","List(Phishing, CyberSecurity, DDoS)",List(mccannkimberly),Reviewing database logs.
5051073293040626730,New phishing campaign detected. Doctor sister owner structure allow. #CyberSecurity #CVE @thomasjones @fwilliams,2024-11-08T13:08:12.000Z,@smclaughlin,753385439,es,4,5,1,0,51,https://example.com/news/86,unknown,Android,false,false,null,1074380751614937220,180504,752,false,South William,Phishing,social_engineering,web_portal,2025-12-26T18:16:01.467Z,"List(New, phishing, campaign, detected., Doctor, sister, owner, structure, allow., #CyberSecurity, #CVE, @thomasjones, @fwilliams)","List(CyberSecurity, CVE)","List(thomasjones, fwilliams)",New phishing campaign detected. Doctor sister owner structure allow.
3820283122683259858,Account concern present expect experience stay throughout large model move morning commercial. #DDoS,2024-12-15T08:00:50.000Z,@garyperez,429061271,en,4,7,2,0,59,unknown,unknown,Android,false,false,null,6976430402798092748,1871,1481,false,West Sheilaberg,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Account, concern, present, expect, experience, stay, throughout, large, model, move, morning, commercial., #DDoS, )",List(DDoS),List(),Account concern present expect experience stay throughout large model move morning commercial.
8373494114952244489,Lawyer area change data hair tonight theory two per. You contain end. #Ransomware #DDoS #DataBreach @wmartinez @julie16,2025-02-17T21:20:29.000Z,@dnelson,872172330,en,3,8,0,2,44,unknown,unknown,Twitter Web App,false,false,null,4724040809333899990,1819,1176,false,Alyssastad,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Lawyer, area, change, data, hair, tonight, theory, two, per., You, contain, end., #Ransomware, #DDoS, #DataBreach, @wmartinez, @julie16)","List(Ransomware, DDoS, DataBreach)","List(wmartinez, julie16)",Lawyer area change data hair tonight theory two per. You contain end.
1967683443240152632,Place those peace interesting house course Republican mouth evening part person practice. #CVE #DataBreach @anthonycastaneda @woodkyle,2024-12-22T12:45:37.000Z,@matthewjones,69631146,en,0,10,1,1,58,unknown,https://img.example.com/3.jpg,Android,false,false,null,9618597534815003393,4300,1994,false,unknown,unknown,unknown,unknown,2025-12-26T18:16:01.467Z,"List(Place, those, peace, interesting, house, course, Republican, mouth, evening, part, person, practice., #CVE, #DataBreach, @anthonycastaneda, @woodkyle)","List(CVE, DataBreach)","List(anthonycastaneda, woodkyle)",Place those peace interesting house course Republican mouth evening part person practice.


### Querying the newly created columns

In [0]:
df = df.drop("tokens")
display(df.select("text", "cleaned_text", "hashtags_array", "mentions_array").limit(5))

text,cleaned_text,hashtags_array,mentions_array
Reviewing database logs. #Phishing #CyberSecurity #DDoS @mccannkimberly,Reviewing database logs.,"List(Phishing, CyberSecurity, DDoS)",List(mccannkimberly)
New phishing campaign detected. Doctor sister owner structure allow. #CyberSecurity #CVE @thomasjones @fwilliams,New phishing campaign detected. Doctor sister owner structure allow.,"List(CyberSecurity, CVE)","List(thomasjones, fwilliams)"
Account concern present expect experience stay throughout large model move morning commercial. #DDoS,Account concern present expect experience stay throughout large model move morning commercial.,List(DDoS),List()
Lawyer area change data hair tonight theory two per. You contain end. #Ransomware #DDoS #DataBreach @wmartinez @julie16,Lawyer area change data hair tonight theory two per. You contain end.,"List(Ransomware, DDoS, DataBreach)","List(wmartinez, julie16)"
Place those peace interesting house course Republican mouth evening part person practice. #CVE #DataBreach @anthonycastaneda @woodkyle,Place those peace interesting house course Republican mouth evening part person practice.,"List(CVE, DataBreach)","List(anthonycastaneda, woodkyle)"


In [0]:
df = df.withColumn(
    "has_hashtags",
    size(col("hashtags_array")) > 0
).withColumn(
    "has_mentions",
    size(col("mentions_array")) > 0
).withColumn(
    "word_count",
    size(split(col("cleaned_text"), " "))
).withColumn(
    "created_date",
    to_date(col("created_at"))
).withColumn(
    "ingest_time",
    current_timestamp()
)


### Creating Row Id (surrogate Key)

In [0]:
from pyspark.sql.window import Window

In [0]:
window_spec = Window.orderBy(
    col("created_at"),
    col("id")
)

df = df.withColumn(
    "row_id",
    row_number().over(window_spec)
)

### selecting the needed columns for silver table

In [0]:
df.columns

['id',
 'text',
 'created_at',
 'username',
 'user_id',
 'language',
 'retweet_count',
 'like_count',
 'reply_count',
 'quote_count',
 'impression_count',
 'urls',
 'media_urls',
 'source',
 'is_retweet',
 'is_reply',
 'in_reply_to_user_id',
 'conversation_id',
 'user_followers_count',
 'user_following_count',
 'user_verified',
 'user_location',
 'attack_type',
 'delivery_method',
 'context_target',
 'ingest_time',
 'hashtags_array',
 'mentions_array',
 'cleaned_text',
 'has_hashtags',
 'has_mentions',
 'word_count',
 'created_date',
 'row_id']

In [0]:
silver_df = df.select(
    'row_id',
    'id',
    'text',
    'cleaned_text',
    'hashtags_array',
    'mentions_array',
    'has_hashtags',
    'has_mentions',
    'word_count',
    'created_at',
    'created_date',
    'username',
    'user_id',
    'user_verified',
    'user_location',
    'language',
    'retweet_count',
    'like_count',
    'reply_count',
    'quote_count',
    'impression_count',
    'user_followers_count',
    'user_following_count',
    'urls',
    'media_urls',
    'source',
    'is_retweet',
    'is_reply',
    'in_reply_to_user_id',
    'conversation_id',
    'attack_type',
    'delivery_method',
    'context_target',
    'ingest_time',
)

### Creating a Silver Table

In [0]:
silver_df.write.format("delta").mode("overwrite").option("delta.enableChangeDataFeed", "true").saveAsTable("sentimental_analysis.silver.social_media_silver")

### Querying the Silver Table

In [0]:
%sql 
select * from sentimental_analysis.silver.social_media_silver
limit 5

row_id,id,text,cleaned_text,hashtags_array,mentions_array,has_hashtags,has_mentions,word_count,created_at,created_date,username,user_id,user_verified,user_location,language,retweet_count,like_count,reply_count,quote_count,impression_count,user_followers_count,user_following_count,urls,media_urls,source,is_retweet,is_reply,in_reply_to_user_id,conversation_id,attack_type,delivery_method,context_target,ingest_time
1,9191903636345700142,Patch for SQL Injection vulnerability released. #CVE #CVE,Patch for SQL Injection vulnerability released.,"List(CVE, CVE)",List(),true,false,7,2024-08-01T00:00:27.000Z,2024-08-01,@sbaldwin,857212553,false,unknown,en,1,14,1,1,33,2003,1626,unknown,unknown,Twitter Web App,false,false,null,3692899713833546847,SQL Injection,exploit,corporate_network,2025-12-29T05:56:43.116Z
2,2768468865306412359,Republican take evening leader week season event program fish Mrs increase common. Such large parent candidate. #Firewall #ZeroDay @david39 @bakerbrent,Republican take evening leader week season event program fish Mrs increase common. Such large parent candidate.,"List(Firewall, ZeroDay)","List(david39, bakerbrent)",true,true,16,2024-08-01T00:01:32.000Z,2024-08-01,@karen16,456484389,true,Lake Mark,en,0,5,0,1,38,19990,1060,https://example.com/news/85,https://img.example.com/1.jpg,Android,false,false,null,7546778863198935481,unknown,unknown,unknown,2025-12-29T05:56:43.116Z
3,6734007271730864259,Guess job miss more eat example can first war base economy approach require. #SOC @serickson,Guess job miss more eat example can first war base economy approach require.,List(SOC),List(serickson),true,true,13,2024-08-01T00:01:39.000Z,2024-08-01,@lisa16,605847713,false,South Angelica,en,1,10,0,4,47,1794,1869,unknown,unknown,TweetDeck,false,true,null,9155366862291657394,unknown,unknown,unknown,2025-12-29T05:56:43.116Z
4,7825357171775709264,Security audit went smoothly. #InfoSec #CyberSecurity,Security audit went smoothly.,"List(InfoSec, CyberSecurity)",List(),true,false,5,2024-08-01T00:02:13.000Z,2024-08-01,@henry28,657894082,false,North Margarettown,en,3,9,0,1,59,22861,1967,unknown,unknown,TweetDeck,false,false,null,6655308818574900253,Man-in-the-Middle,exploit,corporate_network,2025-12-29T05:56:43.116Z
5,1489034316026594924,New ddos campaign detected. #CyberSecurity #MFA #ThreatIntel,New ddos campaign detected.,"List(CyberSecurity, MFA, ThreatIntel)",List(),true,false,5,2024-08-01T00:02:45.000Z,2024-08-01,@brittany66,819843095,false,Lake Megan,en,2,9,0,0,55,835,106,unknown,unknown,Android,false,false,null,8282303825448412450,DDoS,exploit,web_portal,2025-12-29T05:56:43.116Z
